# Домашняя работа №9.2

## Цели и задачи работы:
1) Написать алгоритм, который принимает на вход любую видеопоследовательность;
2) В данной видеопоследовательности необходимо реализовать и зафиксировать на первых кадрах две точки: точку в центре экрана и опорную точку;
3) Вычислить координаты данных точек на каждом кадре видеопоследовательности и рассчитать ошибку определения опорной точки для каждого кадра;
4) Оценить, какой детектор лучше подходит для определения опорных точек на каждом кадре видеопоследовательности;
5) Наложить фильтр шумов на видеопоследовательность и оценить ошибку определения опорной точки для каждого кадра с шумами;
6) Наложить фильтр для повышения контрастности на видеопоследовательность;

## Подготовка

В качестве видеопоследовательности будет использоваться 15-ти секундное видео с полетом дрона.

Для дальнейших манипуляций с видеопоследовательностью импортируем следующие библиотеки:

In [6]:
import cv2;
import numpy as np;

# 1.Реализация алгоритма вычисления координат точек на кадрах видеопотока и ошибки их определения 

Реализуем функцию "track_video_function()", которая принимает на вход следующие аргументы:
1) "input_video" - путь к видеофайлу для обработки;
2) "output_video" - путь, куда будет сохранен видеофайл после обработки.

In [10]:
def track_video_function(input_video, output_video):
    print("Начало обработки видеопотока...");
    capture = cv2.VideoCapture(input_video);
    video_fps = capture.get(cv2.CAP_PROP_FPS);
    video_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH));
    video_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT));
    video_codec = cv2.VideoWriter_fourcc(*'mp4v')
    output = cv2.VideoWriter(output_video, video_codec, video_fps, (video_width, video_height));
    retur, first_frame = capture.read();
    
    if not retur:
        print(f"Не удается прочитать видеофайл {input_video}");
        return;

    detector_type = cv2.ORB_create();
    keypoints, descriptors = detector_type.detectAndCompute(first_frame, None);

    if len(keypoints) > 0:
        initial_point = keypoints[290].pt;
    
    else:
        print("Ключевые точки не найдены");
        return;

    points_to_track = np.array([[initial_point]], dtype = np.float32);
    gray_map_initial = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY);
    center_point = (video_width // 2, video_height // 2);
    output.write(first_frame);
    trajectories = [];

    while True:
        retur, current_frame = capture.read();
        
        if not retur:
            break;

        gray_map_current = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY);
        new_points, point_status, error = cv2.calcOpticalFlowPyrLK(gray_map_initial, gray_map_current, points_to_track, None);

        if point_status[0] == 1:
            tracked_point = new_points[0][0];
            trajectories.append((int(tracked_point[0]), int(tracked_point[1])));

            if len(trajectories) >= 3:
                source_points = np.array(trajectories[-3:], dtype = np.float32);
                destination_points = np.array([points_to_track[0][0], tracked_point, tracked_point], dtype = np.float32);
                M, inliers = cv2.estimateAffinePartial2D(source_points, destination_points);

                if M is not None:
                    points_to_track = cv2.transform(points_to_track, M);

            if len(trajectories) >= 3:
                
                for i in range(2, len(trajectories)):
                    cv2.line(current_frame, trajectories[i - 1], trajectories[i], (255, 0, 0), 2);

            if len(trajectories) > 3:
                cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);

            cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);
            cv2.drawMarker(current_frame, center_point, (0, 255, 0), markerType = cv2.MARKER_CROSS, markerSize = 20, thickness = 2);
            end_point = (center_point[0] + 100, center_point[1] + 100);
            cv2.line(first_frame, center_point, end_point, (255, 0, 0), 2);
            draw_function(current_frame, tracked_point, trajectories, center_point);
            points_to_track = new_points.reshape(-1, 1, 2);           
        
        else:
            break;  

        output.write(current_frame);
        cv2.imshow("Video file point tracking in process...", current_frame);

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break;

        gray_map_initial = gray_map_current;
        
    capture.release();
    output.release();
    cv2.destroyAllWindows();
    print(f"Обработка видеопотока завершена. Обработанный файл {output_video} сохранен в директорию проекта");

Последовательность работы данной функции:
1) При помощи функций "cv2.VideoCapture()", "capture.get()" мы осуществляем чтение видеофайла и получаем ряд его параметров: частоту кадров в секунду, ширину и высоту каждого кадра видеофайла в пикселях; 
2) При помощи функций "cv2.VideoWriter_fourcc()" и "cv2.VideoWriter()" выполняется определение кодека видео после обработки и задаются его выходные параметры (путь к выходному видеофайлу, частота видео (кадров/сек) и т.д.).
3) Конструкция "retur, first_frame = capture.read()" выполняет чтение каждого кадра входного видеофайла. Конструкция: 

    """
    if not retur:
        print(f"Не удается прочитать видеофайл {input_video}");
        return;
    """

возвращает "False", если не было получено значение "True" для параметра "retur", который является маркером чтения кадра.

4) Функция "cv2.ORB_create()" определяет тип детектора "ORB", который будет использоваться по умолчанию для поиска ключевых точек.
Функция "detector_type.detectAndCompute()" будет получать набор ключевых точек и их дескрипторы для каждого кадра видеопотока.
Конструкция:

    """
    if len(keypoints) > 0:
        initial_point = keypoints[290].pt;

    else:
        print("Ключевые точки не найдены");
        return;
    """

определяет начальную точку из набора, которая будет использоваться в качестве ключевой. Конструкция возвращает "False", если ключевые точки не были определены.

5) Конструкция "points_to_track = np.array([[initial_point]], dtype = np.float32)" определяет массив ключевых точек для каждого кадра в формате "np.float32". Функция "cv2.cvtColor()" переводит каждый кадр в цветовое пространство "COLOR_BGR2GRAY" (все цвета на кадре будут являться градациями серого). Конструкция "center_point = (video_width // 2, video_height // 2)" определяет центр каждого кадра. Функция "output.write()" осуществялет запись каждого кадра в выходной видеофайл. Переменная "trajectories" представляет собой пустой массив для хранения координат опорной ключевой точки, определяющих ее траекторию.

6) 1-ая часть конструкции:

    """
    while True:
        retur, current_frame = capture.read();
        
        if not retur:
            break;

        gray_map_current = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY);
        new_points, point_status, error = cv2.calcOpticalFlowPyrLK(gray_map_initial, gray_map_current, points_to_track, None);

        if point_status[0] == 1:
            tracked_point = new_points[0][0];
            trajectories.append((int(tracked_point[0]), int(tracked_point[1])));

            if len(trajectories) >= 3:
                source_points = np.array(trajectories[-3:], dtype = np.float32);
                destination_points = np.array([points_to_track[0][0], tracked_point, tracked_point], dtype = np.float32);
                M, inliers = cv2.estimateAffinePartial2D(source_points, destination_points);

                if M is not None:
                    points_to_track = cv2.transform(points_to_track, M);

        ...
    """

выполняет чтение каждого кадра видеопотока посредством функции "capture.read()". Текущий кадр переводится в цветовое пространство "COLOR_BGR2GRAY" посредством функции "cv2.cvtColor()". Затем, при помощи функции "cv2.calcOpticalFlowPyrLK()", вычисляется оптический поток между ключевыми точками текущего и предыдущего кадра в цветовом пространстве "COLOR_BGR2GRAY" при помощи итеративного метода Лукаса-Канады с использованием пирамид. Если ключевая точка в кадре найдена, то ее координаты звписываются в массив "trajectories". Когда в массиве "trajectories" будут находиться координаты как минимум 3-ех точек, то посредством функции "cv2.estimateAffinePartial2D()" будет вычисляться матрица афинного преобразования. Если матрица афинного преобразования определена, то при ее помощи вычисляется следующий набор ключевых точек для следующего кадра.

7) 2-ая часть конструкции:

    """
        ...
    if len(trajectories) >= 3:
                
        for i in range(2, len(trajectories)):
            cv2.line(current_frame, trajectories[i - 1], trajectories[i], (255, 0, 0), 2);

    if len(trajectories) > 3:
        cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);

    cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);
    cv2.drawMarker(current_frame, center_point, (0, 255, 0), markerType = cv2.MARKER_CROSS, markerSize = 20, thickness = 2);
    end_point = (center_point[0] + 100, center_point[1] + 100);
    cv2.line(first_frame, center_point, end_point, (255, 0, 0), 2);
    draw_function(current_frame, tracked_point, trajectories, center_point);
    points_to_track = new_points.reshape(-1, 1, 2);           
        
    else:
        break;
        
    output.write(current_frame);
    cv2.imshow("Video file point tracking in process...", current_frame);

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break;

    gray_map_initial = gray_map_current;
    """

выполняет отрисовку опорной ключевой точки и точки, определяющей центр изображения, на каждом кадре, а также выполняет вызов функции "draw_function()". Затем выполняется вызов окна "Video file point tracking in process..." для каждого кадра видеопоследовательности. Конструкция "cv2.waitKey(30) & 0xFF == ord('q')" позволяет пользователю прервать цикл выполнения обработки видео посредством нажатия клавиши "Q", при этом уже обработанная часть видеопоследовательности будет сохранена в директорию проекта. Затем происходит замена изображений в цветовом пространстве "COLOR_BGR2GRAY".

Конструкция:

    """
    capture.release();
    output.release();
    cv2.destroyAllWindows();
    print(f"Обработка видеопотока завершена. Обработанный файл {output_video} сохранен в директорию проекта");
    """

выполняет процесс чтения входного видеопотока и записи выходного видеопотока;

Реализуем функцию "draw_function()", которая принимает на вход следующие аргументы:
1) "current_frame" - текущий кадр видеопотока;
2) "tracked_point" - ключевая точка, которая была определена;
3) "trajectories" - массив, хранящий координаты ключевых точек;
4) "center_point" - точка, определяющая центр каждого кадра.

In [29]:
def draw_function(current_frame, tracked_point, trajectories, center_point):
    coordinates_text = f"Координаты точки: X = {int(tracked_point[0])}, Y = {int(tracked_point[1])}";
    error_value = np.linalg.norm(np.array(trajectories[-1]) - np.array(tracked_point));
    error_text = f"Ошибка: {int(error_value)} пк";   
    cv2.putText(current_frame, coordinates_text, (20, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (100, 100, 100), 2);
    cv2.putText(current_frame, error_text, (20, 80), cv2.FONT_HERSHEY_COMPLEX, 1, (100, 100, 100), 2);

Данная функция выполняет отрисовку текстовых данных о координатах опорной точки на каждом кадре видеопотока и ошибке его определения.

Выполним вызов функции "track_video_function()":

In [33]:
track_video_function('Input_video.mp4', 'Output_video_ORB.mp4');

Начало обработки видеопотока...
Обработка видеопотока завершена. Обработанный файл Output_video_ORB.mp4 сохранен в директорию проекта


Результат работы алгоритма с детектором "ORB":

<video controls src="Output_video_ORB.mp4"/>

Как можно увидеть, средняя ошибка определения ключевой точки составляет 1 пиксель.

Теперь реализуем обработку данного видео при помощи детектора "SIFT":

In [42]:
def track_video_function(input_video, output_video):
    print("Начало обработки видеопотока...");
    capture = cv2.VideoCapture(input_video);
    video_fps = capture.get(cv2.CAP_PROP_FPS);
    video_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH));
    video_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT));
    video_codec = cv2.VideoWriter_fourcc(*'mp4v')
    output = cv2.VideoWriter(output_video, video_codec, video_fps, (video_width, video_height));
    retur, first_frame = capture.read();
    
    if not retur:
        print(f"Не удается прочитать видеофайл {input_video}");
        return;

    detector_type = cv2.SIFT_create();
    keypoints, descriptors = detector_type.detectAndCompute(first_frame, None);

    if len(keypoints) > 0:
        initial_point = keypoints[6700].pt;
    
    else:
        print("Ключевые точки не найдены");
        return;

    points_to_track = np.array([[initial_point]], dtype = np.float32);
    gray_map_initial = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY);
    center_point = (video_width // 2, video_height // 2);
    output.write(first_frame);
    trajectories = [];

    while True:
        retur, current_frame = capture.read();
        
        if not retur:
            break;

        gray_map_current = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY);
        new_points, point_status, error = cv2.calcOpticalFlowPyrLK(gray_map_initial, gray_map_current, points_to_track, None);

        if point_status[0] == 1:
            tracked_point = new_points[0][0];
            trajectories.append((int(tracked_point[0]), int(tracked_point[1])));

            if len(trajectories) >= 3:
                source_points = np.array(trajectories[-3:], dtype = np.float32);
                destination_points = np.array([points_to_track[0][0], tracked_point, tracked_point], dtype = np.float32);
                M, inliers = cv2.estimateAffinePartial2D(source_points, destination_points);

                if M is not None:
                    points_to_track = cv2.transform(points_to_track, M);

            if len(trajectories) >= 3:
                
                for i in range(2, len(trajectories)):
                    cv2.line(current_frame, trajectories[i - 1], trajectories[i], (255, 0, 0), 2);

            if len(trajectories) > 3:
                cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);

            cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);
            cv2.drawMarker(current_frame, center_point, (0, 255, 0), markerType = cv2.MARKER_CROSS, markerSize = 20, thickness = 2);
            end_point = (center_point[0] + 100, center_point[1] + 100);
            cv2.line(first_frame, center_point, end_point, (255, 0, 0), 2);
            draw_function(current_frame, tracked_point, trajectories, center_point);
            points_to_track = new_points.reshape(-1, 1, 2);           
        
        else:
            break;  

        output.write(current_frame);
        cv2.imshow("Video file point tracking in process...", current_frame);

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break;

        gray_map_initial = gray_map_current;
        
    capture.release();
    output.release();
    cv2.destroyAllWindows();
    print(f"Обработка видеопотока завершена. Обработанный файл {output_video} сохранен в директорию проекта");

def draw_function(current_frame, tracked_point, trajectories, center_point):
    coordinates_text = f"Координаты точки: X = {int(tracked_point[0])}, Y = {int(tracked_point[1])}";
    error_value = np.linalg.norm(np.array(trajectories[-1]) - np.array(tracked_point));
    error_text = f"Ошибка: {int(error_value)} пк";   
    cv2.putText(current_frame, coordinates_text, (20, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (100, 100, 100), 2);
    cv2.putText(current_frame, error_text, (20, 80), cv2.FONT_HERSHEY_COMPLEX, 1, (100, 100, 100), 2);

Выполним обработку видеопотока при помощи детектора "SIFT":

In [45]:
track_video_function('Input_video.mp4', 'Output_video_SIFT.mp4');

Начало обработки видеопотока...
Обработка видеопотока завершена. Обработанный файл Output_video_SIFT.mp4 сохранен в директорию проекта


Результат работы алгоритма с детектором "SIFT":

<video controls src="Output_video_SIFT.mp4"/>

Здесь видно одну их основных проблем детектора "SIFT". Он имеет крайне большой дрейф ключевых точек при обработке кадров видеопотока, что описано в статье "A Comparison of SIFT, SURF and ORB on OpenCV" (https://mikhail-kennerley.medium.com/a-comparison-of-sift-surf-and-orb-on-opencv-59119b9ec3d0). Приведем иллюстрацию из данной статьи:

![title](https://miro.medium.com/v2/resize:fit:1004/format:webp/1*tiR4AZSt-ltVlYv_3ds6Yw.png)

Это приводит к тому, что опорная ключевая точка на объекте при его исчезновении из определенного кадра видеопотока плавно смещается в "поле определения" ближайшего объекта и алгоритм считает, что опорная точка не исчезла из поля зрения, что является проблемой. Также, по времени выполнения алгоритма, видно, что детектор "SIFT" работает медленнее детектора "ORB", что также проиллюстрированно в статье "A Comparison of SIFT, SURF and ORB on OpenCV":

![title](https://miro.medium.com/v2/resize:fit:1010/format:webp/1*cf44qgjFB-tjNiG1Nn5n7A.png)

В связи с вышеперечисленными недостатками детектора "SIFT", в дальнейшем мы будем использовать детектор "ORB".

## 2. Добавление фильтра шумов в видеопоток

Теперь реализуем функцию "distortion_noise_function()", которая принимает на вход следующие аргументы:
1) "frame" - текущий кадр;
2) "distortion_level" - уровень шума.

In [11]:
def distortion_noise_function(frame, distortion_level=0.65):
    noise = np.random.normal(0, 25, frame.shape).astype(np.uint8);
    distorted_frame = cv2.addWeighted(frame, 1 - distortion_level, noise, distortion_level, 0);
    return distorted_frame;

Данная функция генерирует случайный шум. Шум создается с использованием нормального распределения с матожиданием "0" и стандартным отклонением "25", что приводит к случайным значениям яркости, варьирующимся вдоль пикселей. Результирующий массив преобразуется в тип "uint8", что соответствует типу данных, используемому для представления пикселей изображений в "OpenCV".

Теперь модифицируем код, добавив вычисление отклонения координат опорной ключевой точки между кадром без искажений и кадром с добавлением случайного шума вдоль осей "X" и "Y". Также добавим отображение данных отклонений на каждый кадр видеопотока:

In [16]:
# Блок импорта библиотек

import cv2;
import numpy as np;

# Блок функций

def track_video_function(input_video, output_video):
    print("Начало обработки видеопотока...");
    capture = cv2.VideoCapture(input_video);
    video_fps = capture.get(cv2.CAP_PROP_FPS);
    video_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH));
    video_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT));
    video_codec = cv2.VideoWriter_fourcc(*'mp4v');
    output = cv2.VideoWriter(output_video, video_codec, video_fps, (video_width, video_height));
    retur, first_frame = capture.read();
    first_frame = distortion_noise_function(first_frame);
    
    if not retur:
        print(f"Не удается прочитать видеофайл {input_video}");
        return;

    detector_type = cv2.ORB_create();
    keypoints, descriptors = detector_type.detectAndCompute(first_frame, None);

    if len(keypoints) > 0:
        initial_point = keypoints[240].pt;
    else:
        print("Ключевые точки не найдены");
        return;

    points_to_track = np.array([[initial_point]], dtype=np.float32);
    gray_map_initial = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY);
    center_point = (video_width // 2, video_height // 2);
    output.write(first_frame);
    trajectories = [];
    original_point = points_to_track[0][0];
    total_error_x = 0;
    total_error_y = 0;
    frame_count = 0;

    while True:
        retur, current_frame = capture.read();
        current_frame = distortion_noise_function(current_frame);
        
        if not retur:
            break;
        
        gray_map_current = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY);
        new_points, point_status, error = cv2.calcOpticalFlowPyrLK(gray_map_initial, gray_map_current, points_to_track, None);

        if point_status[0] == 1:
            tracked_point = new_points[0][0];
            trajectories.append((int(tracked_point[0]), int(tracked_point[1])));
            difference = np.array(tracked_point) - np.array(original_point);
            original_point = tracked_point;
            error_value_x = difference[0];
            error_value_y = difference[1];
            total_error_x += error_value_x;
            total_error_y += error_value_y;
            frame_count += 1;
            
            if len(trajectories) >= 3:
                source_points = np.array(trajectories[-3:], dtype=np.float32);
                destination_points = np.array([points_to_track[0][0], tracked_point, tracked_point], dtype=np.float32);
                M, inliers = cv2.estimateAffinePartial2D(source_points, destination_points);

                if M is not None:
                    points_to_track = cv2.transform(points_to_track, M);

            if len(trajectories) >= 3:
                for i in range(2, len(trajectories)):
                    cv2.line(current_frame, trajectories[i - 1], trajectories[i], (255, 0, 0), 2);

            if len(trajectories) > 3:
                cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);

            cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);
            cv2.drawMarker(current_frame, center_point, (0, 255, 0), markerType=cv2.MARKER_CROSS, markerSize=20, thickness=2);
            end_point = (center_point[0] + 100, center_point[1] + 100);
            cv2.line(first_frame, center_point, end_point, (255, 0, 0), 2);
            draw_function(current_frame, tracked_point, trajectories, center_point, difference, total_error_x, total_error_y, frame_count);
            points_to_track = new_points.reshape(-1, 1, 2);
            
        else:
            break;

        output.write(current_frame);
        cv2.imshow("Video file point tracking in process...", current_frame);

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break;

        gray_map_initial = gray_map_current;

    capture.release();
    output.release();
    cv2.destroyAllWindows();
    print(f"Обработка видеопотока завершена. Обработанный файл {output_video} сохранен в директорию проекта");

def draw_function(current_frame, tracked_point, trajectories, center_point, difference, total_error_x, total_error_y, frame_count):
    coordinates_text = f"Координаты точки: X = {int(tracked_point[0])}, Y = {int(tracked_point[1])}";
    error_value = np.linalg.norm(np.array(trajectories[-1]) - np.array(tracked_point));
    error_text = f"Ошибка: {int(error_value)} пк";
    difference_text = f"Смещение опорной ключевой точки с учетом шумов: dX = {int(difference[0])}, dY = {int(difference[1])}";
    average_error_x_text = f"Средняя ошибка смещения с учетом шума: dX = {float(total_error_x / frame_count) if frame_count > 0 else 0} пк";
    average_error_y_text = f"Средняя ошибка смещения с учетом шума: dY = {float(total_error_y / frame_count) if frame_count > 0 else 0} пк";
    cv2.putText(current_frame, coordinates_text, (20, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2);
    cv2.putText(current_frame, error_text, (20, 80), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2);
    cv2.putText(current_frame, difference_text, (20, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2);
    cv2.putText(current_frame, average_error_x_text, (20, 160), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2);
    cv2.putText(current_frame, average_error_y_text, (20, 200), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2);

Теперь выполним вычисление отклонения координат опорной ключевой точки между кадром без искажений и кадром с добавлением случайного шума, c уровнем "0.65", вдоль осей "X" и "Y":

In [83]:
track_video_function('Input_video.mp4', 'Output_video_(noise).mp4');

Начало обработки видеопотока...
Обработка видеопотока завершена. Обработанный файл Output_video_(noise).mp4 сохранен в директорию проекта


Результат обработки видеопотока с шумами:

<video controls src="Output_video_(noise).mp4"/>

По результатам обработки получены следующие результаты:

dX = -0.71 пикселя;
dY = 1.42 пикселя.

Это средняя величина ошибки координат опорной ключевой точки между кадром без искажений и кадром с добавлением случайного шума, c уровнем "0.65", вдоль осей "X" и "Y" c учетом направления смещения относительно номинального положения (с учетом знака). Абсолютная средняя величина ошибки будет больше.

## 3. Добавление фильтра резкости

Теперь реализуем функцию "sobel_filter_function()", которая принимает на вход следующие аргументы:
1) "frame" - текущий кадр;
2) "alpha" - коэффициент перекрытия текущего кадра фильтром.

In [101]:
def sobel_filter_function(frame, alpha=1.5):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]]);
    filter_add = cv2.filter2D(frame, -1, kernel);
    filtered_frame = cv2.addWeighted(frame, 1 - alpha, filter_add, alpha, 0);
    return filtered_frame;

В данной функции создается матрица (ядро) "kernel" для фильтра Собеля, предназначенная для увеличения резкости изображения. 
Центральный элемент матрицы повышает яркость пикселя, в то время как соседние элементы уменьшают её, что выделяет края объектов на изображении. Затем данный фильтр применяется к текущему кадру при помощи функций "cv2.filter2D()", "cv2.addWeighted()".

Теперь модифицируем код, добавив фильтр резкости:

In [116]:
# Блок импорта библиотек

import cv2;
import numpy as np;

# Блок функций

def track_video_function(input_video, output_video):
    print("Начало обработки видеопотока...");
    capture = cv2.VideoCapture(input_video);
    video_fps = capture.get(cv2.CAP_PROP_FPS);
    video_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH));
    video_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT));
    video_codec = cv2.VideoWriter_fourcc(*'mp4v');
    output = cv2.VideoWriter(output_video, video_codec, video_fps, (video_width, video_height));
    retur, first_frame = capture.read();
    first_frame = sobel_filter_function(first_frame);
    
    if not retur:
        print(f"Не удается прочитать видеофайл {input_video}");
        return;

    detector_type = cv2.ORB_create();
    keypoints, descriptors = detector_type.detectAndCompute(first_frame, None);

    if len(keypoints) > 0:
        initial_point = keypoints[360].pt;
    else:
        print("Ключевые точки не найдены");
        return;

    points_to_track = np.array([[initial_point]], dtype=np.float32);
    gray_map_initial = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY);
    center_point = (video_width // 2, video_height // 2);
    output.write(first_frame);
    trajectories = [];

    while True:
        retur, current_frame = capture.read();
        current_frame = sobel_filter_function(current_frame);
        
        if not retur:
            break;
        
        gray_map_current = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY);
        new_points, point_status, error = cv2.calcOpticalFlowPyrLK(gray_map_initial, gray_map_current, points_to_track, None);

        if point_status[0] == 1:
            tracked_point = new_points[0][0];
            trajectories.append((int(tracked_point[0]), int(tracked_point[1])));

            if len(trajectories) >= 3:
                source_points = np.array(trajectories[-3:], dtype=np.float32);
                destination_points = np.array([points_to_track[0][0], tracked_point, tracked_point], dtype=np.float32);
                M, inliers = cv2.estimateAffinePartial2D(source_points, destination_points);

                if M is not None:
                    points_to_track = cv2.transform(points_to_track, M);

            if len(trajectories) >= 3:
                for i in range(2, len(trajectories)):
                    cv2.line(current_frame, trajectories[i - 1], trajectories[i], (255, 0, 0), 2);

            if len(trajectories) > 3:
                cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);

            cv2.circle(current_frame, (int(tracked_point[0]), int(tracked_point[1])), 10, (0, 0, 255), -1);
            cv2.drawMarker(current_frame, center_point, (0, 255, 0), markerType=cv2.MARKER_CROSS, markerSize=20, thickness=2);
            end_point = (center_point[0] + 100, center_point[1] + 100);
            cv2.line(first_frame, center_point, end_point, (255, 0, 0), 2);
            draw_function(current_frame, tracked_point, trajectories, center_point);
            points_to_track = new_points.reshape(-1, 1, 2);
            
        else:
            break;

        output.write(current_frame);
        cv2.imshow("Video file point tracking in process...", current_frame);

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break;

        gray_map_initial = gray_map_current;

    capture.release();
    output.release();
    cv2.destroyAllWindows();
    print(f"Обработка видеопотока завершена. Обработанный файл {output_video} сохранен в директорию проекта");

def draw_function(current_frame, tracked_point, trajectories, center_point):
    coordinates_text = f"Координаты точки: X = {int(tracked_point[0])}, Y = {int(tracked_point[1])}";
    error_value = np.linalg.norm(np.array(trajectories[-1]) - np.array(tracked_point));
    error_text = f"Ошибка: {int(error_value)} пк";
    cv2.putText(current_frame, coordinates_text, (20, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2);
    cv2.putText(current_frame, error_text, (20, 80), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2);
    
def sobel_filter_function(frame, alpha=1.5):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]]);
    filter_add = cv2.filter2D(frame, -1, kernel);
    filtered_frame = cv2.addWeighted(frame, 1 - alpha, filter_add, alpha, 0);
    return filtered_frame;

Выполним обработку изображения применив фильтр резкости:

In [119]:
track_video_function('Input_video.mp4', 'Output_video_(sobel_filter).mp4');

Начало обработки видеопотока...
Обработка видеопотока завершена. Обработанный файл Output_video_(sobel_filter).mp4 сохранен в директорию проекта


Результат обработки видеопотока с фильтром резкости:

<video controls src="Output_video_(sobel_filter).mp4"/>

## Выводы:

В результате работы были выполнены все цели и задачи. Было установлено, что применение детектора "ORB" для видеопотока предпочтительнее, чем детектора "SIFT", поскольку детектор "ORB" имеет лучшее быстродействие, а также почти не подвержен дрейфу ключевых точек. Также было установлено, что применение фильтра шумов к видеопотоку ухудшает точность определения координат ключевых точек для каждого кадра.